# Stereo Visual Odometry
This notebook apply stereo depth estimation and multiple view geometry to track vehicle position through a sequencee of the images from kitti dataset.

In [ ]:
import os
import cv2
import pandas as pd


class DatasetHandler(object):
    def __init__(self):
        pass

First, let's get the ground truth trajectory

In [ ]:
poses = pd.read_csv("../dataset/poses/00.txt", delimiter=' ', header=None)

$$
\begin{bmatrix}
    x_w \\ y_w \\ z_w
\end{bmatrix}
$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

image = plt.imread()
plt.imshow(image, cmap='gray')
plt.show()


## Camera Projection and Calibration


In [ ]:
calib = pd.read_csv('', delimiter=' ', header=None, index_col=0)
calib

In calibration file, we can obtain four `3x4` projection matrices for four position camera: `p0`, `p1`, `p2` and `p3`. And we can also obtain the transformation matrix for the LiDAR that denoted as `Tr`.

In [ ]:
# P0 is the left grayscale camera
import numpy as np

p0: np.ndarray = np.array(calib.loc['P0:']).reshape((3, 4))

A projection matrix project three dimensional cooridnates in the global coordinates (world coordinates) onto two dimensional image plane (pixel coordinate)

$$
\begin{bmatrix}
    u \\ v \\ 1
\end{bmatrix} = \frac{1}{\lambda}P
\begin{bmatrix}
    x_w \\ y_w\\ z_w\\ 1
\end{bmatrix}
$$

where $\lambda$ is known as the scale, which is the depth to the point along z-axis from the camera. The projection matrix `P` is the dot product of intrinsic matrix and extrinsic matrix. The intrinsic matrix `K` that describe the focal length and optical center parameters of camera, and the extrinsic matrix is the augmented matrix of rotation matrix and translation vector that describe the transformation between world coordinates and camera coordinates

$$
\begin{bmatrix}
    u \\ v \\ 1
\end{bmatrix} = \frac{1}{\lambda}P
\begin{bmatrix}
    x_w \\ y_w\\ z_w\\ 1
\end{bmatrix} = \frac{1}{\lambda}K[R|t]
\begin{bmatrix}
    x_w \\ y_w\\ z_w\\ 1
\end{bmatrix}
$$

In [ ]:
p1: np.ndarray = np.array(calib.loc['P1:']).reshape((3, 4))

import cv2
k, r, t, _, _, _, _ = cv2.decomposeProjectionMatrix(p1)


## Dataset Handler
Implement the dataset handler to manipulate the dataset from files, and make it more accessible to complete our mission.

In [ ]:
class DatasetHandler(object):
    def __init__(self) -> None:
        pass

    def _reset(self) -> None:
        pass

## Stereo Depth Estimation



<div align='center'>
    <img src="../assets/stereo.png" width='300' alt='stereo' />
</div>

With similar triangles, we can derive as following:

$$
\frac{Z}{f}=\frac{X}{x_L}, \frac{Z}{f}=\frac{X-b}{x_R}
$$

and we define **disparity `d`** as the difference between $x_L$ amd $x_R$, which means the difference in horizontal pixel location of the point projected onto left and right image plane.

$$
d = (x_L - x_R)
$$

Thus, we can get

$$
fb=Zd \rightarrow Z=\frac{fb}d
$$

In [ ]:
import cv2
import numpy as np

def compute_disparity_map(
    image_l: np.ndarray,
    image_r: np.ndarray,
    matcher: str = 'bm',
    rgb_map: bool = False,
    verbose: bool = False,
) -> np.ndarray | None:
    if matcher not in ('bm', 'sgbm'):
        raise ValueError('matcher type is not in list.')
    
    if matcher == 'bm':
        matcher = cv2.StereoBM_create()
    else:
        print('sgbm')

    if rgb_map:
        l = cv2.cvtColor(image_l, cv2.COLOR_BGR2GRAY)
        r = cv2.cvtColor(image_r, cv2.COLOR_BGR2GRAY)

    ## computing matching
    import datetime
    t0 = datetime.datetime.now()
    disp = matcher.compute(l, r).astype(np.float32) / 16
    t1 = datetime.datetime.now()

    if verbose:
        print(f'Time to compute disparity map with stereo{matcher}: {t1 -t0}')
    

## LiDAR Dataset Process
Process the LiDAR data from the dataset, which could be used for depth estimation and odometry correction later.

## Feature Extraction and Matching
Extract the feature from image and matching between each frame to find the transformation relationship in camera poses between a sequences of frames.

## Visual Odometry


In [ ]:
class VO(object):
    def __init__(self) -> None:
        pass

In [ ]:
class CalcError(object):
    def __init__(self, gt: np.ndarray, est: np.ndarray) -> None:
        pass

    @staticmethod
    def _mse(gt: np.ndarray, est: np.ndarray) -> np.float64:
        n: int = est.shape[0]
        se = np.power(np.sqrt(
            np.power(gt[n, 0, 3] - est[:, 0, 3], 2) +  
            np.power(gt[n, 1, 3] - est[:, 1, 3], 2) +
            np.power(gt[n, 2, 3] - est[:, 2, 3], 2) 
        ), 2)
        mse = np.mean(se)
        return mse